In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import flayer

In [ ]:
input_columns = ["F-layer thickness", "ICB delta T", "Gruneisen parameter", "XFe top", 
                 "XFe ICB", "Nucleation rate", "Initial radius", "Chemical diffusivity",
                 
                ]

In [7]:
df = pd.DataFrame(columns=input_columns)
df[1] = df[1].astype(object)
df.loc[1, 1] = np.array([5, 6, 7, 8])
df.loc[2, 1] = np.array([[5, 6], [7, 8]])
df

,1
1,"[5, 6, 7, 8]"
2,"[[5, 6], [7, 8]]"


In [8]:
a = df.loc[2, 1]

In [9]:
a

array([[5, 6],
       [7, 8]])